# Deal with Multi-omics/Multi-view scCCC

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import muon as mu
import liana as li

/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## li.mt.multiview(MuData, x_mod, y_mod, interactions, transform_fun=None, ...)

In [2]:
from itertools import product

In [3]:
from anndata import concat

In [4]:
from scipy.sparse import csr_matrix, isspmatrix_csr

In [6]:
# TODO: check this makes sense + add zero-inflated filtering as in CiteFuse
def zi_minmax(X, cutoff=0.1):
    
    # Ensure the matrix is in Compressed Sparse Row (CSR) format
    if not isspmatrix_csr(X):
        X = X.tocsr()
    # Min-Max scaling on non-zero elements
    min_vals = np.array(X.min(axis=0).todense())[0]
    max_vals = np.array(X.max(axis=0).todense())[0]
    nonzero_rows, nonzero_cols = X.nonzero()
    scaled_values = (X.data - min_vals[nonzero_cols]) / (max_vals[nonzero_cols] - min_vals[nonzero_cols])
    
    # apply cutoff
    scaled_values[scaled_values < cutoff] = 0

    # Create a new sparse matrix with scaled values
    X = csr_matrix((scaled_values, (nonzero_rows, nonzero_cols)), shape=X.shape)
    
    return X

In [7]:
## Refactor slightly liana pipe to accept interactions (i.e. list of tuples?)

In [8]:
# Then this just becomes a wrapper around the liana pipe, where the method of interest is passed as an argument? + We do zi_minmax to deal with different scales
# Then change select_resource to return tuples instead of a dataframe, and dataframe when df=True

In [9]:
# We take the two modalities and stack as values to the same AnnData
# transform_fun will be None and it can be applied by columns (i.e. by variable); e.g. lambda x: zi_minmax(x, axis=0)

In [10]:
from liana.testing._sample_anndata import generate_toy_mdata
mdata = generate_toy_mdata()
mdata.mod['adata_y'].var.index = mdata.mod['adata_y'].var.index + '_y'
sc.pp.scale(mdata.mod['adata_y'])

/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/mudata/_core/mudata.py:458: UserWarning: Cannot join columns with the same name because var_names are intersecting.


In [11]:
interactions = list(product(mdata.mod['adata_x'].var.index, mdata.mod['adata_y'].var.index))

In [12]:
resource = pd.DataFrame(interactions, columns=['ligand', 'receptor'])

Actual function

In [13]:
mod_x = 'adata_x'
mod_y = 'adata_y'

In [14]:
mdata.mod[mod_x].X = mdata.mod[mod_x].X.astype(np.float32)
mdata.mod[mod_y].X = mdata.mod[mod_x].X.astype(np.float32)

In [15]:
# Concat the two modalities
adata = concat([mdata.mod['adata_x'], mdata.mod['adata_y']], join='outer', axis=1, merge='first')

Apply minmax

In [16]:
adata.layers['raw'] = adata.X.copy()

In [17]:
adata.X = zi_minmax(adata.X)

In [18]:
adata.X.A[:,3][0:20]

array([0.4637132 , 0.        , 0.        , 0.49868843, 0.37161177,
       0.32148063, 0.43398425, 0.53366363, 0.7222384 , 0.43340135,
       0.        , 0.38822502, 0.38501894, 0.        , 0.7225299 ,
       0.6607403 , 0.        , 0.        , 0.6275139 , 0.        ],
      dtype=float32)

In [19]:
adata.layers['raw'].A[:,3][0:20]

array([1.591, 0.   , 0.   , 1.711, 1.275, 1.103, 1.489, 1.831, 2.478,
       1.487, 0.   , 1.332, 1.321, 0.   , 2.479, 2.267, 0.   , 0.   ,
       2.153, 0.   ], dtype=float32)

In [ ]:
adata.X = csr_matrix(MinMaxScaler.fit_transform(adata.X.todense()))

In [ ]:
li.mt.cellphonedb(adata, resource=resource, groupby='bulk_labels', use_raw=False)

## li.fun.estimate_metalinks(adata, est_fun ...) 
returns (metabolite_estimates:csr_matrix, receptors_masked:csr_matrix)

## li.mt.metalinks(adata, groupby, resource, ...)